# DESaster Simulation Set Up Template

Last edited: Sept 27th 2016

## Required Modules

In [1]:
import sys, random
desaster_path = "/Users/Derek/Dropbox/Simulation/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

C:\Users\Derek\Anaconda2\envs\py34k\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


## Input Data

In [2]:
#scenario_file = '../inputs/scenario_test1.xlsx'
scenario_file = "C:/Users/Derek/Desktop/desaster_inputs9262016.xlsx"
# Create Pandas dataframe of attribute data for all households to be modeled in the simulation
# required column names, exactly as written: Name , Savings , Insurance
households_df = pd.read_excel(scenario_file, sheetname='households')

# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
housing_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

If all houses are completely damaged, we have the possibility of this much damage:

In [3]:
households_df["Value"].sum()

162031921.53999999

yet we only budgeted 2 million for building material cost. We might need to do a delayed deposit of more materials as infrustructure is rebuild

In [4]:
housing_stock_df.head()

,ID Number,Value,Area,Damage State,Latitude,Longitude,Parcel_ID,Occupancy
0,US001269,59100.0,864.0,Slight,46.584940,-123.906278,13102783101,Single Family Dwelling
1,US001270,20600.0,440.0,Moderate,46.584875,-123.905274,13102783102,Single Family Dwelling
2,US001271,23400.0,660.0,Slight,46.694724,-123.652259,14081470000,Single Family Dwelling
3,US001272,28100.0,957.0,Slight,46.679248,-123.721380,14081944068,Single Family Dwelling
4,US001273,29100.0,804.0,Moderate,46.679371,-123.721384,14081944069,Single Family Dwelling


we need to shuffle the households dataframe before we feed it into the sim

In [5]:
households_df = households_df.sample(frac=1).reset_index(drop=True)
#takes a random sample, frac is a fraction to sample (1 means take a 100% sample), reset index just resets the index to count from 0 tp 1


In [6]:
households_df

,ID Number,Name,Value,Area,Content_Value,Damage State,Latitude,Longitude,Parcel_ID,Occupancy,Has Insurance,Savings,Insurance
0,US000672,MABEL BURTON,124200.0,2436.00,62100.0,Slight,46.346079,-124.052144,73011087005,Single Family Dwelling,0,4499.333240,0
1,US000067,VASHTI SOLORZANO,16600.0,672.00,8300.0,Extensive,46.494028,-124.042399,76016000005,Mobile Home,0,2118.722905,0
2,US000447,MICKIE MARK,39075.8,1300.00,19537.9,Extensive,46.359212,-124.053428,73011040003,Mobile Home,0,2964.024941,0
3,US000534,INGEBORG SHELDON,46300.0,1059.09,23150.0,Slight,46.364114,-124.053623,74060214000,Single Family Dwelling,0,3220.267224,0
4,US002729,LADY LOFTON,31500.0,1248.00,15750.0,Complete,46.668516,-123.828769,14092942021,Mobile Home,0,2699.292958,0
5,US002840,BETTINA BROUSSARD,57100.0,1532.00,28550.0,Moderate,46.371672,-123.762116,10091121074,Mobile Home,0,3520.930810,0
6,US002353,ALINA GOSS,56700.0,1080.00,28350.0,Extensive,46.448352,-124.049011,74042000062,Mobile Home,0,3512.496941,0
7,US000114,ARDITH HENLEY,47900.0,936.00,23950.0,Extensive,46.749887,-124.085806,78006003005,Mobile Home,0,3282.393352,0
8,US000984,SUK WHIPPLE,32000.0,780.00,16000.0,Slight,46.694283,-123.742205,72064001003,Single Family Dwelling,0,2723.875002,0
9,US000086,BERNA BOHANNON,11700.0,924.00,5850.0,Extensive,46.742003,-124.084485,78003000019,Mobile Home,0,1870.241043,0


In [7]:
human_cap_data["Engineers"] = 40

In [8]:
human_cap_data

Type
Contractors             40
Engineers               40
FEMA Processors         40
Inspectors             100
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [9]:
financial_cap_data

Type
Building Materials     2000000
FEMA Aid              35000000
Name: Quantity, dtype: int64

## Simulation Initiation

__Set simulation environment__

In [10]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [11]:
write_story = True

__Populate the simulation with the input data specified above.__

In [12]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

human_capital = capitals.HumanCapital(simulation, human_cap_data)

households = entities.importHouseholds(simulation, households_df, write_story)

housing_stock = capitals.importHousingStock(simulation, housing_stock_df)

__Write a master process that combines process and functions from search, rebuild, and request modules.__

In [13]:
def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 1000  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
               
   

In [14]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], write_story))

__Do some cool stuff with the vacant housing stock.__

In [15]:
# Do inspections on all of the vacant homes in the housing stock
for home in housing_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

# start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

In [16]:
start_delayed(simulation, capitals.reloadBuildingMaterial(simulation, financial_capital.building_materials, amount = 100000000), 100)
simulation.run()

## Outputs

__Summary statistics__

In [17]:
num_undamaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_gave_up_home_search = 0

for household in households:
    if household.money_search_start == 0.0: num_undamaged += 1
    if household.home_get > 0.0: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1
    if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} households suffered no damage to their homes.\n'.format(num_undamaged, len(households)),
      '{0} out of {1} households rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(households)),
        '{0} out of {1} households gave up searching for money.\n'.format(num_gave_up_money_search, len(households)),
        '{0} out of {1} households decided to find a new home.\n'.format(num_relocated, len(households)),
        '{0} out of {1} households gave up searching for a home.'.format(num_gave_up_home_search, len(households))
      )

46 out of 2860 households suffered no damage to their homes.
 2759 out of 2860 households rebuilt or repaired their damaged home.
 0 out of 2860 households gave up searching for money.
 3 out of 2860 households decided to find a new home.
 55 out of 2860 households gave up searching for a home.


__Household stories__

In [18]:
len(households)

2860

In [19]:
building_mat_insuff = 0
for i in households:
    if "There were insufficient construction materials available in the area for {0} to rebuild. ".format(i.household['Name']) in i.story:
        building_mat_insuff += 1
print(building_mat_insuff)

0


In [20]:
vars(financial_capital.building_materials)

{'_capacity': inf,
 '_env': <simpy.core.Environment at 0x735ae90>,
 '_level': 76014706.354700118,
 'get': <bound method Container.ContainerGet of <simpy.resources.container.Container object at 0x08EDE150>>,
 'get_queue': [],
 'put': <bound method Container.ContainerPut of <simpy.resources.container.Container object at 0x08EDE150>>,
 'put_queue': []}

In [27]:
vars(households[0])

{'assessment_get': 145.0,
 'assessment_put': 120.0,
 'assistance_get': 0.0,
 'assistance_payout': 0.0,
 'assistance_put': 0.0,
 'assistance_request': 0.0,
 'claim_get': 0.0,
 'claim_payout': 0.0,
 'claim_put': 0.0,
 'gave_up_home_search': False,
 'gave_up_money_search': False,
 'home_get': 182.0,
 'home_put': 180.0,
 'home_search_start': 0.0,
 'home_search_stop': 0.0,
 'household': ID Number                      US000672
 Name                       MABEL BURTON
 Value                            124200
 Area                               2436
 Content_Value                     62100
 Damage State                     Slight
 Latitude                        46.3461
 Longitude                      -124.052
 Parcel_ID                   73011087005
 Occupancy        Single Family Dwelling
 Has Insurance                         0
 Savings                         4499.33
 Insurance                             0
 Name: 0, dtype: object,
 'inspection_get': 120.0,
 'inspection_put': 0,
 'insuranc

In [30]:
rebilt = []
for i in households:
    rebilt.append(i.home_get)

In [35]:
rebilt.sort()

In [40]:
for i in households:
    print("{0}......{1}".format(i.household.Name, i.home_get))

MABEL BURTON......182.0
VASHTI SOLORZANO......260.0
MICKIE MARK......260.0
INGEBORG SHELDON......182.0
LADY LOFTON......0.0
BETTINA BROUSSARD......240.0
ALINA GOSS......260.0
ARDITH HENLEY......260.0
SUK WHIPPLE......182.0
BERNA BOHANNON......260.0
MICAELA MAYES......182.0
EARLINE GAINES......260.0
FALLON DUPREE......260.0
LAURA LOPEZ......207.0
ANDREW WHITMORE......260.0
PROVIDENCIA GAITHER......260.0
LATOYIA SEVERSON......260.0
REBBECCA OH......260.0
SHENITA BURDETTE......260.0
ANGLEA CUELLAR......260.0
MEE CLEARY......260.0
PANDORA RAWLS......182.0
MINNA GIRARD......182.0
IZOLA LILES......260.0
SHERIDAN CONTI......260.0
STACIE SHEPHERD......270.0
GENNY COLBURN......182.0
LIANE MARRERO......270.0
JACINDA KUNTZ......270.0
FERNANDE SHULL......190.0
ELBA COOLEY......270.0
DELPHINE GIBBONS......250.0
DONYA DOE......270.0
NADA HAY......270.0
ISELA GAGE......270.0
CORAZON SCHILLING......270.0
TAM BARONE......182.0
ARDELLE HARGIS......270.0
MELLIE SWANN......182.0
GEORGETTE FRYE......270.0


In [43]:
vars(households[4])

{'assistance_get': 140.0,
 'assistance_payout': 31499.999999999993,
 'assistance_put': 120.0,
 'assistance_request': 31499.999999999993,
 'claim_get': 0.0,
 'claim_payout': 0.0,
 'claim_put': 0.0,
 'gave_up_home_search': True,
 'gave_up_money_search': False,
 'home_get': 0.0,
 'home_put': 0.0,
 'home_search_start': 140.0,
 'home_search_stop': 0.0,
 'household': ID Number           US002729
 Name             LADY LOFTON
 Value                  31500
 Area                    1248
 Content_Value          15750
 Damage State        Complete
 Latitude             46.6685
 Longitude           -123.829
 Parcel_ID        14092942021
 Occupancy        Mobile Home
 Has Insurance              0
 Savings              2699.29
 Insurance                  0
 Name: 4, dtype: object,
 'inspection_get': 120.0,
 'inspection_put': 0,
 'insurance': 0,
 'loan_amount': 0.0,
 'loan_get': 0.0,
 'loan_put': 0.0,
 'money_search_start': 120.0,
 'money_search_stop': 140.0,
 'money_to_rebuild': 34199.292957695594,
